# Import dependencies

In [ ]:
# For typing
from typing import TypeAlias

# For Data science
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from scipy import stats
from sklearn import (
    linear_model,
    model_selection,
    preprocessing,
)

# For modules
from sources import (
    get_heatmap,
    evaluate_model,
)

Typing

In [ ]:
# Get new names for long object types
LinearRegressionModel: TypeAlias = linear_model.LinearRegression
StandardScaler: TypeAlias = preprocessing.StandardScaler

# Getting data, observations
Get datasets

In [ ]:
# Get datasets
red_wine = pd.read_csv(
    filepath_or_buffer="../data/winequality-red.csv",
    delimiter=";",
)
white_wine = pd.read_csv(
    filepath_or_buffer="../data/winequality-white.csv",
    delimiter=";",
)

Show dataset info

In [ ]:
# Get data for red wine
red_wine.info(verbose=False)

In [ ]:
# Get data for white wine
white_wine.info(verbose=False)

Look for missing values

In [ ]:
if white_wine.isna().any().any() | white_wine.isna().any().any():
    print("There are missing values in datasets")
else:
    print("There is no missing values in datasets")

Look for correlations

In [ ]:
get_heatmap(
    dataframe=red_wine,
    name="red_wine",
);

In [ ]:
get_heatmap(
    dataframe=white_wine,
    name="white_wine",
)

There is the sufficient correlation between density and residual sugar features. Let's get the combined feature instead of correlating ones.

In [ ]:
# Update correlating feature
white_wine = white_wine.assign(
    density_to_residuals=white_wine["residual sugar"] * white_wine["density"]
)

# Drop original columns
white_wine.drop(
    columns=["density", "residual sugar"],
    inplace=True,
)

# Move assigned column to the first position
new_column_order = ["density_to_residuals"] + [
    column for column in white_wine if column != "density_to_residuals"
]

white_wine_ordered = white_wine.reindex(columns=new_column_order)

# Get heatmap for updated dataset
get_heatmap(
    dataframe=white_wine_ordered,
    name="white_wine",
)

Now, there are no correlating features in dataset. Let's look at target distribution.

In [ ]:
# Get distribution for red wine target
sns.countplot(data=red_wine, x="quality");

In [ ]:
# Get distribution for white wine target
sns.countplot(data=white_wine, x="quality");

Target distributions looked as normal ones. Let's check the normality by test.

In [ ]:
red_wine_normality = stats.shapiro(red_wine["quality"])[0]
white_wine_normality = stats.shapiro(white_wine["quality"])[0]

print(
    f"Normality test statistics for the both dataset targets are:\
    {red_wine_normality:.2f} and {white_wine_normality:.2f}."
)

Not a perfect result but the acceptable one.

# Linear regression for red_wine dataset
Splitting dataframe to train and test subsets

In [ ]:
# Set feature and target subsets
red_wine_features = red_wine.iloc[:, :-1]
red_wine_target = red_wine.iloc[:, -1]

# Get split subsets
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    red_wine_features,
    red_wine_target,
    test_size=0.3,
    random_state=42,
    shuffle=True,
    stratify=red_wine_target,
)

Scale features

In [ ]:
# Get scaler
red_wine_scaler = preprocessing.StandardScaler(with_mean=True)

# Train scaler
red_wine_X_train_scaled = red_wine_scaler.fit_transform(X_train)

# Scale test data
red_wine_X_test_scaled = red_wine_scaler.transform(X_test)

Check if split target data is balanced

In [ ]:
# Get DataFrame from Series object
train_df = pd.DataFrame(y_train, columns=["quality"])

# Create a count plot for the DataFrame column
plt.figure(figsize=(8, 6))
sns.countplot(x="quality", data=train_df)
plt.title("Count plot for train_df")

# Get portions
y_train.value_counts(normalize=True)

Repeat for test subset

In [ ]:
# Get DataFrame from Series object
test_df = pd.DataFrame(y_test, columns=["quality"])

# Create a count plot for the DataFrame column
plt.figure(figsize=(8, 6))
sns.countplot(x="quality", data=test_df)
plt.title("Count plot for test_df")

# Get portions
y_test.value_counts(normalize=True)

Figures and portion values show balansed status.

Get linear regression

In [ ]:
# Get regression object
red_wine_reg = linear_model.LinearRegression()

# Get fit
red_wine_reg.fit(
    X=red_wine_X_train_scaled,
    y=y_train,
)

# Get train score
red_wine_train_score = red_wine_reg.score(
    X=red_wine_X_train_scaled,
    y=y_train,
)

# Geet test score
red_wine_test_score = red_wine_reg.score(
    X=red_wine_X_test_scaled,
    y=y_test,
)

print(f"Train score: {red_wine_train_score:.2f}.")
print(f"Test score: {red_wine_test_score:.2f}.")

Make cross-validation

In [ ]:
# Get cross-validation results
cross_validation_score = model_selection.cross_val_score(
    estimator=red_wine_reg,
    X=red_wine_X_train_scaled,
    y=y_train,
    cv=5,
)

print(f"Cross validation score: {cross_validation_score[:]}.")

Model looks stable

Let's evaluate model

In [ ]:
# Get metrics for red wine model
evaluate_model(
    model=red_wine_reg,
    x_train=red_wine_X_train_scaled,
    x_test=red_wine_X_test_scaled,
    y_train=y_train,
    y_test=y_test,
)

Trained and test metrics look quite similar. Let's visualize errors.

In [ ]:
# Get density plot
sns.kdeplot(y_test, fill=True, color="r", label="test subset")

sns.kdeplot(
    red_wine_reg.predict(red_wine_X_test_scaled),
    fill=True,
    color="b",
    label="predicted",
)

plt.title("Distribution of observations in test dataset and and predicted dataset")
plt.legend();

There are regions under 4 and above 8 quality grade which is not covered by model. It's suspected that the reason is the presence of out-layers in the dataset.

# Linear regression for white_wine dataset
Splitting dataframe to train and test subsets

In [ ]:
# Set feature and target subsets
white_wine_features = white_wine_ordered.iloc[:, :-1]
white_wine_target = white_wine_ordered.iloc[:, -1]

# Get split subsets
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    white_wine_features,
    white_wine_target,
    test_size=0.3,
    random_state=100,
)

Scale features

In [ ]:
# Get scaler
white_wine_scaler = preprocessing.StandardScaler()

# Train scaler
white_wine_X_train_scaled = white_wine_scaler.fit_transform(X_train)
# Scale test data
# NOTE: use the same mean and standard deviation based on the training data - transform()
white_wine_X_test_scaled = white_wine_scaler.transform(X_test)

Check if split target data is balanced

In [ ]:
# Get DataFrame from Series object
train_df = pd.DataFrame(y_train, columns=["quality"])

# Create a count plot for the DataFrame column
plt.figure(figsize=(8, 6))
sns.countplot(x="quality", data=train_df)
plt.title("Count plot for train_df")

# Get portions
y_train.value_counts(normalize=True)

Repeat for test subset

In [ ]:
# Get DataFrame from Series object
test_df = pd.DataFrame(y_test, columns=["quality"])

# Create a count plot for the DataFrame column
plt.figure(figsize=(8, 6))
sns.countplot(x="quality", data=test_df)
plt.title("Count plot for test_df")
plt.show()

# Get portions
y_test.value_counts(normalize=True)

Figures and portion values show not quite balanced status.

Get linear regression

In [ ]:
# Get regression object
white_wine_reg = linear_model.LinearRegression()

# Get fit
white_wine_reg.fit(
    X=white_wine_X_train_scaled,
    y=y_train,
)

# Get train score
white_wine_train_score = white_wine_reg.score(
    X=white_wine_X_train_scaled,
    y=y_train,
)

# Geet test score
white_wine_test_score = white_wine_reg.score(white_wine_X_test_scaled, y_test)

print(f"Train score: {white_wine_train_score:.2f}.")
print(f"Test score: {white_wine_test_score:.2f}.")

In [ ]:
# Get cross-validation results
cross_validation_score = model_selection.cross_val_score(
    estimator=white_wine_reg,
    X=white_wine_X_train_scaled,
    y=y_train,
    cv=5,
)

print(f"Cross validation score: {cross_validation_score[:]}.")

Model looks stable

Let's evaluate model

In [ ]:
# Evaluate model
evaluate_model(
    model=white_wine_reg,
    x_train=white_wine_X_train_scaled,
    x_test=white_wine_X_test_scaled,
    y_train=y_train,
    y_test=y_test,
)

Trained and test metrics look quite similar. Let's visualize errors.

In [ ]:
# Get density plot
sns.kdeplot(y_test, fill=True, color="r", label="test subset")

sns.kdeplot(
    white_wine_reg.predict(white_wine_X_test_scaled),
    fill=True,
    color="b",
    label="predicted",
)

plt.title("Distribution of observations in test dataset and and predicted dataset.")

plt.legend();

There are regions under 4 and above 8 quality grade which is not covered by model. It's suspected that the reason is the presence of out-layers in the dataset.

# Summary
1. Both datasets were analysed and correlating features were combined to new feature.
2. Both regression models (for red and white wine) were successfully trained.
3. Values of the coefficient of determination for both models are relatively low (0,24 - 0.38) that would come from the out-layers in datasets.
4. Out-layer detection and cleaning is required to get models improved.